In [259]:
import numpy as np
import matplotlib.pyplot as plt

$ l $ x $ w $ board, spaces enumerated sequentially from $ \mathbb{N} $, starting at $ NW $ corner and moving right and down like reading a book.

$ SE $ spot has reward $ \texttt{rewar} $, every other space has reward $ \texttt{cost} $.

actions are north, west, south, east, enumerated $ ( 0, 1, 2, 3 ) $, taking at action has probability $ \texttt{fail} $ to fail and randomly choose any direction to go

if agent takes an action that would go off the board it remains in the same space

exception is if agent is in goal state (SE), then $ \mathcal{A} $ corresponds to a teleportation to the four corners of the board $ ( 0 : NE, \; 1 : NW, \; 2 : SW, \; 3 : SE ) $ with the normal fail likelihood

In [260]:
width = 4
height = 4
rewar = 1
cost = 0
fail = 0.1
epsilo = 0.1
rho = 0.5
gamm = 0.8
toleran = 0.1

In [261]:
goalState = width * height - 1

# reward function that returns reward variable if agent is in the SE space
# and return cost variable otherwise
def reward (state):
    
    if state == goalState:
        return rewar
    else:
        return cost

# given state is SE-most, check if on the border,
# and then get new state or don't change state accordingly
# if state is SE-most, then return the corresponding corner state
def move (state, choice):

    new = state
    if state != goalState:
        if choice == 0:
            if state >= width:
                new = state - width
        elif choice == 1:
            if state % width != 0:
                new = state - 1
        elif choice == 2:
            if state < width * (height - 1):
                new = state + width
        else:
            if state % width != width - 1:
                new = state + 1

    else:
        if choice == 0:
            new = width - 1
        elif choice == 1:
            new = 0
        elif choice == 2:
            new = width * (height - 1)
    return new

# simulate if chosen action is taken or fails
# if fail randomly generate chosen action
def action (state, choice):

    if np.random.random() < (1 - fail):
        return move (state, choice)
    else:
        return move (state, np.random.randint(0, 4))

# get list of indices of the max value within an 1d numpy array
def argQmax (Q_X):

    maxim = max(Q_X)
    ind = []

    for i in range(0, len(Q_X)):
        if maxim == Q_X[i]:
            ind.append(i)

    return ind[np.random.randint(0, len(ind))]

In [262]:
# Q-learning method from pseudocode from unified two timescale approach to mean field game/control
def QRL (X, A, epsilon, tol_Q, rho_Q, gamma, episodes, ep_len):
    
    Q = [np.zeros([len(X), len(A)])]

    for k in range(1, episodes + 1):
        Q.append(Q[k - 1].copy())
        state = X[0]
        for n in range(0, ep_len):
            if np.random.random() > epsilon:
                choic = argQmax(Q[k][state])
            else:
                choic = np.random.randint(0, len(A))
            newState = action(state, choic)
            r = reward(newState)
            # do we need to worry about epsilon-greedy policy for Q values
            Q[k][state][choic] += rho_Q * (r + gamma * Q[k][newState][argQmax(Q[k][newState])] - Q[k][state][choic])
            state = newState
        
        if np.absolute(np.subtract(Q[k], Q[k - 1])).sum() < tol_Q:
            print(k)
            break
    
    return Q[k]

In [263]:
# get value by getting expected value,
# considering the epsilon-greedy policy on the actions based on 1d Q_matrix row
def getV (Q_X, epsilon):

    maxInd = argQmax(Q_X)
    v = Q_X[maxInd] * (1 - epsilon)

    for i in range(0, 4):
        v += Q_X[i] * epsilon / 4

    return v

# using helper getV method, collect the state-values of each space on the board,
# transforming the state space into a 2d list
def getVmat (Q_mat, epsilon):
    
    V = []
    for h in range(0, height):
        V.append([])
        for w in range(0, width):
            V[h].append(getV(Q_mat[h * width + w], epsilon))

    return V

# seed, Q learn, get state-values matrix, print Q-matrix
np.random.seed(10)

Q_XA = QRL(list(range(0, width * height)), [0, 1, 2, 3], epsilo, toleran, rho, gamm, 100, 1000)
V_mat = getVmat(Q_XA, epsilo)

print("the Q Matrix composed of state-action-values")
print("")
print("               north    west   south    east")
print("")
for i in range(0, width * height):
    print("state %3d:    %6.3f  %6.3f  %6.3f  %6.3f " % (i, Q_XA[i][0], Q_XA[i][1], Q_XA[i][2], Q_XA[i][3]))

the Q Matrix composed of state-action-values

               north    west   south    east

state   0:     0.979   0.870   0.977   1.095 
state   1:     1.022   1.123   1.310   1.357 
state   2:     1.546   1.149   1.215   1.744 
state   3:     1.568   1.333   2.606   1.493 
state   4:     0.897   1.003   1.273   1.090 
state   5:     1.119   1.013   1.580   1.462 
state   6:     1.318   1.426   1.656   2.018 
state   7:     1.685   1.651   3.282   2.159 
state   8:     1.056   1.243   1.700   1.199 
state   9:     1.400   1.303   1.859   1.615 
state  10:     1.738   1.695   3.092   2.097 
state  11:     2.223   2.166   3.665   2.595 
state  12:     1.332   1.686   1.671   2.159 
state  13:     1.840   1.725   2.053   2.673 
state  14:     2.095   1.942   2.166   3.618 
state  15:     1.744   0.947   1.628   4.384 


In [264]:
print("board with calculated state-values on the epsilon-greedy policy")
print("")

for h in range(0, height):
    for w in range(0, width):
        print("+ - - - - ", end = '')
    print("+")
    for w in range(0, width):
        print("|         ", end = '')
    print("|")
    for w in range(0, width):
        print("| %7.4f " % (V_mat[h][w]), end = '')
    print("|")
    for w in range(0, width):
        print("|         ", end = '')
    print("|")

for w in range(0, width):
    print("+ - - - - " % (V_mat[h][w]), end = '')
print("+")

board with calculated state-values on the epsilon-greedy policy

+ - - - - + - - - - + - - - - + - - - - +
|         |         |         |         |
|  1.0833 |  1.3413 |  1.7110 |  2.5200 |
|         |         |         |         |
+ - - - - + - - - - + - - - - + - - - - +
|         |         |         |         |
|  1.2519 |  1.5513 |  1.9763 |  3.1728 |
|         |         |         |         |
+ - - - - + - - - - + - - - - + - - - - +
|         |         |         |         |
|  1.6595 |  1.8272 |  2.9986 |  3.5646 |
|         |         |         |         |
+ - - - - + - - - - + - - - - + - - - - +
|         |         |         |         |
|  2.1148 |  2.6131 |  3.5016 |  4.1635 |
|         |         |         |         |
+ - - - - + - - - - + - - - - + - - - - +
